In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")
# df = pd.read_csv("./dataset/A4Benchmark/test.csv") 

#print data
df['timestamps'] = pd.to_datetime(df['timestamps'], unit='s')
df

# standard deviation

In [ ]:
def sliding_window_std(lst, window_size):
    results = []
    
    for i, num in enumerate(lst):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(lst), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = lst[start_index:end_index]
        
        # Calculate the average of the window
        avg = sum(window) / len(window)
        
        # Calculate the squared differences from the average
        squared_diffs = [(avg - n) ** 2 for n in window]
        
        # Calculate the sum of squared differences
        sum_squared_diffs = sum(squared_diffs)
        
        # Calculate the standard deviation of the window
        std_dev = (sum_squared_diffs / len(window)) ** 0.5
        
        # Append the result to the results list
        results.append((num, window, std_dev))
        
    return results

lst = [509.276632, 557.008664, 619.458683, 623.929433, 633.684138, 653.559915, 566.842374, 774.303327, 531.359225, 452.040120,
       472.528354, 491.361270, 539.037251, 591.125394]
window_size = 5

std_devs = sliding_window_std(lst, window_size)
for num, window, std_dev in std_devs:
    print(f"{num}, {window} -> {std_dev}")

# converting the lst into datafram

In [ ]:
import pandas as pd

def sliding_window_std(df, column_name, window_size):
    results = []
    
    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = df[column_name][start_index:end_index]
        
        # Calculate the average of the window
        avg = window.mean()
        
        # Calculate the squared differences from the average
        squared_diffs = [(avg - n) ** 2 for n in window]
        
        # Calculate the sum of squared differences
        sum_squared_diffs = sum(squared_diffs)
        
        # Calculate the standard deviation of the window
        std_dev = (sum_squared_diffs / len(window)) ** 0.5
        
        # Append the result to the results list
        results.append((df.iloc[i][column_name], window.tolist(), std_dev))
        
    return results

# Read data from CSV file into a dataframe
# df = pd.read_csv("./dataset/A4Benchmark/test.csv")
df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")

# Specify the column name containing the data
column_name = 'value'  # Replace 'your_column_name' with the actual column name

window_size = 5

std_devs = sliding_window_std(df, column_name, window_size)
i = 0
for num, window, std_dev in std_devs:
    print(f"{i},{num}, {window} -> {std_dev}")
    i = i+1


# Final

In [ ]:
import pandas as pd

def sliding_window_outliers(df, column_name, window_size, threshold):
    outliers = []
    
    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = df[column_name][start_index:end_index]
        
        # Calculate the average of the window
        avg = window.mean()
        
        # Calculate the standard deviation of the window
        std_dev = window.std()
        
        # Calculate the range based on the threshold
        lower_bound = avg - (threshold * std_dev)
        upper_bound = avg + (threshold * std_dev)

        print("Lower bound:",lower_bound)
        print("Upper bound:",upper_bound)

        # Check for outliers within the window
        for num in window:
            print(i, " : ",num)
            # i +=1
            if num < lower_bound or num > upper_bound:
                outliers.append((i, num))
                
    return outliers

def actual_anomaly(df):
    anomaly_rows = []

    # Iterate over each row in the DataFrame
    for i, row in df.iterrows():
        if row['anomaly'] == 1:
            # Append a tuple containing the row number and the value from the 'anomaly' column to the result list
            anomaly_rows.append((i, row['value']))
    # print(anomaly_rows)
    return anomaly_rows


if __name__ == "__main__":
    # df = pd.read_csv("./dataset/A4Benchmark/test.csv")
    df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")
    
    # Specify the column name containing the data
    column_name = 'value'
    
    window_size = 6
    threshold = 2  # Adjust the threshold as needed (e.g., 1, 2, 3)
    
    outliers = sliding_window_outliers(df, column_name, window_size, threshold)
    actual = actual_anomaly(df)

In [ ]:
for i, outlier in outliers:
    print(f"Index: {i}, Outlier: {outlier}")

In [ ]:
len(outliers)

# Actual Anomalies 

In [ ]:
print(actual)

In [ ]:
len(actual)

In [ ]:
# Extract values from outliers list
outliers_values = set([value for _, value in outliers])

# Find missing values in actual list
missing_values = [(index, value) for index, value in actual if value not in outliers_values]

# Print missing values
if missing_values:
    print("Missing values in outliers list:")
    for index, value in missing_values:
        print(f"Index: {index}, Value: {value}")
else:
    print("All values in the actual list exist in the outliers list.")

In [ ]:
# removing the duplicate.
final_result = []
my_set = set()

for i in outliers:
    my_set.add(i[1])

for i in outliers:
    if i[1] in my_set:
        final_result.append(i)
        my_set.remove(i[1])

In [ ]:
final_result

In [ ]:
len(final_result)

In [ ]:
actual

In [ ]:
# removing the garbage values
final_result_2 = []

for i in final_result:
    for j in actual:
        if i[1] == j[1]:
            final_result_2.append(i)
            continue

In [ ]:
final_result_2

In [ ]:
len(final_result_2)

# ======================================================================

# final final

In [ ]:
def sliding_window_std(lst, window_size, threshold):
    results = []
    stored_values = []  # List to store values that exceed the threshold

    for i, num in enumerate(lst):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(lst), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = lst[start_index:end_index]
        # Calculate the average of the window
        avg = sum(window) / len(window)
        
        # Calculate the squared differences from the average
        squared_diffs = [(avg - n) ** 2 for n in window]
        
        # Calculate the sum of squared differences
        sum_squared_diffs = sum(squared_diffs)
        
        # Calculate the standard deviation of the window
        std_dev = (sum_squared_diffs / len(window)) ** 0.5
        
        # Append the result to the results list
        results.append((num, window, std_dev))

        if len(results) >= 2:
            avg_prev_results = (results[-2][2] + results[-1][2]) / 2
            threshold_value = avg_prev_results + threshold

            if std_dev > threshold_value:
                stored_values.append((i - window_size // 2, window[-1]))
                
        # print('stored values:', stored_values)
    return results, stored_values

# lst = [4, 5, 6, 7, 9, 10, 8, 120, 14, 15, 17, 2, 19, 20, 21, 18, 17]
lst = [509.276632, 557.008664, 619.458683, 623.929433, 633.684138, 653.559915, 566.842374, 774.303327, 531.359225, 
       452.040120, 472.528354, 491.361270, 539.037251, 591.125394]

window_size = 5
threshold = 5  # Adjust threshold as needed

std_devs, anomalies = sliding_window_std(lst, window_size, threshold)
for num, window, std_dev in std_devs:
    print(f"Number: {num}, Window: {window}, Standard Deviation: {std_dev}")
print("########################################################")
print("Anomalies:")
for win_num, value in anomalies:
    print(f"Window number: {win_num}, Value: {value}")

# another test

In [ ]:
def find_normal_values(original_lst, anomaly_lst):
    normal_values = []
    for num in original_lst:
        is_anomaly = False
        for win_num, value in anomaly_lst:
            if num == value:
                is_anomaly = True
                break
        if not is_anomaly:
            normal_values.append(num)
    return normal_values

# Your existing code
def sliding_window_std(lst, window_size, threshold):
    results = []
    stored_values = []  # List to store values that exceed the threshold

    for i, num in enumerate(lst):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(lst), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = lst[start_index:end_index]
        # Calculate the average of the window
        avg = sum(window) / len(window)
        
        # Calculate the squared differences from the average
        squared_diffs = [(avg - n) ** 2 for n in window]
        
        # Calculate the sum of squared differences
        sum_squared_diffs = sum(squared_diffs)
        
        # Calculate the standard deviation of the window
        std_dev = (sum_squared_diffs / len(window)) ** 0.5
        
        # Append the result to the results list
        results.append((num, window, std_dev))

        if len(results) >= 2:
            avg_prev_results = (results[-2][2] + results[-1][2]) / 2
            threshold_value = avg_prev_results + threshold

            if std_dev > threshold_value:
                stored_values.append((i - window_size // 2, window[-1]))
                
    return results, stored_values

# Test data
lst = [509.276632, 557.008664, 619.458683, 623.929433, 633.684138, 653.559915, 566.842374, 774.303327, 531.359225, 
       452.040120, 472.528354, 491.361270, 539.037251, 591.125394]

window_size = 5
threshold = 5  # Adjust threshold as needed

# Get anomalies and standard deviations
std_devs, anomalies = sliding_window_std(lst, window_size, threshold)

# Print anomalies
print("Anomalies:")
for win_num, value in anomalies:
    print(f"Window number: {win_num}, Value: {value}")

# Find normal values
normal_values = find_normal_values(lst, anomalies)

# Print normal values
print("Normal Values:")
print(normal_values)

# into df

In [ ]:
import pandas as pd

def find_normal_values(original_lst, anomaly_lst):
    normal_values = []
    for num in original_lst:
        is_anomaly = False
        for win_num, value in anomaly_lst:
            if num == value:
                is_anomaly = True
                break
        if not is_anomaly:
            normal_values.append(num)
    return normal_values

def sliding_window_std(df, column_name, window_size, threshold):
    results = []
    stored_values = []  # List to store values that exceed the threshold

    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = df[column_name].iloc[start_index:end_index]
        
        # Calculate the standard deviation of the window
        std_dev = window.std()
        
        # Append the result to the results list
        results.append((df.iloc[i][column_name], window.tolist(), std_dev))

        if len(results) >= 2:
            avg_prev_results = (results[-2][2] + results[-1][2]) / 2
            threshold_value = avg_prev_results + threshold

            if std_dev > threshold_value:
                stored_values.append((i - window_size // 2, window.iloc[-1]))
                
    return results, stored_values

# Read data from CSV file into a dataframe
# df = pd.read_csv("./dataset/A4Benchmark/test.csv")
df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")

# Specify the column name containing the data
column_name = 'value'  # Replace 'your_column_name' with the actual column name

window_size = 5
threshold = 10  # Adjust threshold as needed

std_devs, anomalies = sliding_window_std(df, column_name, window_size, threshold)

# Print anomalies
print("Anomalies:")
for win_num, value in anomalies:
    print(f"Window number: {win_num}, Value: {value}")

# Find normal values
normal_values = find_normal_values(df[column_name], anomalies)

# Print normal values
print("Normal Values:")
print(normal_values)


# another one

In [57]:
import pandas as pd

def find_normal_values(original_lst, anomaly_lst):
    normal_values = []
    for num in original_lst:
        is_anomaly = False
        for win_num, value in anomaly_lst:
            if num == value:
                is_anomaly = True
                break
        if not is_anomaly:
            normal_values.append(num)
    return normal_values

In [58]:
def sliding_window_std(df, column_name, window_size, threshold):
    results = []
    problem_vals = []  # List to store values that exceed the threshold

    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
        
        # Get the numbers within the window
        window = df[column_name].iloc[start_index:end_index]
        
        # Calculate the standard deviation of the window
        std_dev = window.std()
        
        # Append the result to the results list
        # result list conations 3 things (number, window,std of the window)
        results.append((i,df.iloc[i][column_name], window.tolist(), std_dev))
        print(results[i],'\n')

        if len(results) <= 2:
            # print("Skipping iteration#", i)
            # print("##################################################")
            continue
            
        if len(results) >= 3:
            # print("val1:",results[-3][3])
            # print("val2:",results[-2][3])
            # print("current std:",std_dev)
            # print("current iteration:", i)
            # print("##################################################")

            avg_prev_results = (results[-2][3] + results[-1][3]) / 2
            threshold_value = avg_prev_results + threshold

            if std_dev > threshold_value:
                problem_vals.append((i - window_size // 2, window.iloc[-1]))

            # print('Problem list:', problem_vals)
    return results, problem_vals

In [59]:
# Read data from CSV file into a dataframe
# df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")
df = pd.read_csv("./dataset/A4Benchmark/test.csv")

# Specify the column name containing the data
column_name = 'value'

window_size = 5
threshold = 10  # Adjust threshold as needed

std_devs, anomalies = sliding_window_std(df, column_name, window_size, threshold)

# # Print anomalies
# print("Anomalies:")
# for row, win_num, value in anomalies:
#     # print(f"Window number: {win_num}, Value: {value}")
#     print(f"Row no: {row}, Value: {value}")

# # Find normal values
# normal_values = find_normal_values(df[column_name], anomalies)

# # Print normal values
# print("Normal Values:")
# print(normal_values)

(0, 2.88922459441453, [2.88922459441453, 112.492219013977, 131.097055735302], 69.27746663599947) 

(1, 112.492219013977, [2.88922459441453, 112.492219013977, 131.097055735302, 128.208835929804], 61.07138115024973) 

(2, 131.097055735302, [2.88922459441453, 112.492219013977, 131.097055735302, 128.208835929804, 121.639029429034], 54.348118668122154) 

(3, 128.208835929804, [112.492219013977, 131.097055735302, 128.208835929804, 121.639029429034, 114.006116387519], 8.282950160211199) 

(4, 121.639029429034, [131.097055735302, 128.208835929804, 121.639029429034, 114.006116387519, 74.1205617662663], 23.144936538964878) 

(5, 114.006116387519, [128.208835929804, 121.639029429034, 114.006116387519, 74.1205617662663, 28.8655901542432], 41.743537091858336) 

(6, 74.1205617662663, [121.639029429034, 114.006116387519, 74.1205617662663, 28.8655901542432, 7.52893206158677], 50.526804531948244) 

(7, 28.8655901542432, [114.006116387519, 74.1205617662663, 28.8655901542432, 7.52893206158677, -30.181247

In [60]:
def finding_row_number(original_lst, anomaly_lst):
    rows = []
    i = 1
    for num in original_lst:
        i +=1
        for win_num, value in anomaly_lst:
            if num == value:
                rows.append((i,num))
    return rows

anomaly_row_list = finding_row_number(df[column_name], anomalies)

In [61]:
print(anomaly_row_list)

[(15, 101.537147158843), (21, -20.3040598349879), (26, 98.888466201236), (33, 130.503216853067), (45, 72.0289989506959), (50, 199.04219908005), (51, 245.243711771896)]


In [62]:
print(len(anomaly_row_list))

7


# model

In [ ]:
anomaly_row_list

# Categorize anomalies
seasonal_12 = []
anomalies = []

for num in anomaly_row_list:
    # print(num[0])
    if num[0] % 12 == 0:  # If index is divisible by 12, consider as seasonal anomaly
        seasonal_12.append(num)
    else:
        anomalies.append(num)

In [ ]:
actual_anomalies

# Categorize anomalies
seasonal_168 = []
actual_anomalies_2 = []

for num in anomaly_row_list:
    if num[0] % 168 == 0:
        seasonal_anomalies_168.append(num)
    else:
        anomalies.append(num)

# model test

In [ ]:
anomaly_row_list

In [ ]:
# Categorize anomalies
seasonal = []
anomalies = []

for num in anomaly_row_list:
    if num[0] % 12 == 0: #or num[0] % 168 == 0:  # If index is divisible by 12, consider as seasonal anomaly
        seasonal.append(num)
    else:
        anomalies.append(num)

In [ ]:
print(seasonal)

In [ ]:
print(anomalies)

# testing by rimsha

In [ ]:
import pandas as pd

def find_normal_values(original_lst, anomaly_lst):
    normal_values = []
    for num in original_lst:
        is_anomaly = False
        for win_num, value in anomaly_lst:
            if num == value:
                is_anomaly = True
                break
        if not is_anomaly:
            normal_values.append(num)
    return normal_values

def sliding_window_std(df, column_name, window_size, threshold):
    results = []

    for i in range(len(df)):
        # Calculate the window boundaries
        start_index = max(0, i - window_size // 2)
        end_index = min(len(df), i + window_size // 2 + 1)
 
        # Get the numbers within the window
        window = df[column_name].iloc[start_index:end_index]
        
        # Calculate the standard deviation of the window
        std_dev = window.std()
        
        # Append the result to the results list
        results.append((df.iloc[i][column_name], window.tolist(), std_dev))
    return results

def test(results, window_size):
    print(len(results))
    stored_values = []
    count = 0
    for i in range(len(results)):
        if  i <= len(results)-2:
            avg_prev_results = (results[i][2] + results[i+1][2]) / 2
            threshold_value = avg_prev_results + threshold
            
        if i <= len(results)-3:
            std = results[i+2][2]
        
        else:
            continue

        if std > threshold_value:
            test= i - window_size // 2
            stored_values.append((abs(test), results[i+2][-2][-1]))

    return stored_values

# Read data from CSV file into a dataframe
# df = pd.read_csv("./dataset/A4Benchmark/test.csv")
df = pd.read_csv("./dataset/A4Benchmark/A4Benchmark-TS1.csv")

# Specify the column name containing the data
column_name = 'value'  # Replace 'your_column_name' with the actual column name

window_size = 7
threshold = 10  # Adjust threshold as needed

std_devs = sliding_window_std(df, column_name, window_size, threshold)

# hello = test(std_devs, window_size)
# print(len(hello))
# print(hello)

# # Print anomalies
print("Anomalies:")
for win_num, value in anomalies:
    print(f"Window number: {win_num}, Value: {value}")
print(len(anomalies))

# Find normal values
normal_values = find_normal_values(df[column_name], anomalies)

# Print normal values
print("Normal Values:")
# print(normal_values)
print(len(normal_values))

kitny asal mian anomaly hy un ko alag kar do... in a list[tuple].


ye jo outliers list hy is ko check karna hy ye seasonal point hy ky nhi?

ghalati sy ye koi asal main outlier miss na hu

supervisor ky golden words :)

seasonla points

minus anomaly then seasional and the remianig will be normal